# Advanced topics in scRNAseq

What we'll cover:

 * Classifying cell types with a reference atlas with `TransferAnchors` from `Seurat`
 * Controlling for Type I error in estimated clusters with `clusterpval`
 * Gene set enrichment analysis with `fgsea`
 * Joint estimation of differential expression and gene set enrichment with `iDEA`
 
(Joselynn: pyscenic? multimodal inference?)

# Gene set enrichment analysis

Gene set enrichment analysis attempts to identify genes that are over-represented in a set of genes that make up pathways associated with various mechanisms. You can think of it like this: if a particular pathway consists of 300 genes, and 280 of those 300 genes are in our list of differentially expressed genes (DEGs), then is that pathway relevant to the differences between our groups of interest?

One of the more basic approaches to answering this question is Overrepresentation Analysis (ORA) (citation: https://yulab-smu.top/biomedical-knowledge-mining-book/references.html#ref-boyle2004). It takes as input a list of DEGs, and a pathway / set of genes, computes the number of DEGs that are found in the pathway, and then computes a p-value using a hypergeometric distribution.

## fgsea

We also run a more standard GSEA analysis, on just the Hallmark gene sets. There is an open question of how to create a ranked list for this analysis. We started with `avg_logFC` as was suggested in [issue#50](https://github.com/ctlab/fgsea/issues/50) on the `fgsea` repo, but it returned no significant pathways at all, which does not seem right for our comparisons also given Table \@ref(tab:de-markers). This may be because in the discussion `avg_logFC` was suggested when comparing markers just between cell type clusters, but we are also actually comparing between different treatment types. Another option was to use `-sign(avg_log2FC)*log10(p_val_adj)`, which incorporates both logfoldchange and the pvalue. This approach seemed more reasonable and yielded results more similar to `iDEA`.

## iDEA